In [1]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# <font color='orange'>01. load data

In [2]:
data_rootpath = r'C:\Users\w10\Desktop\신용평가모형 세미나\2주차\데이터\dev_data'

In [3]:
os.listdir(data_rootpath)

['model_dev_cb_df.csv',
 'model_dev_inner_df.csv',
 'model_valid_cb_df.csv',
 'model_valid_inner_df.csv']

In [4]:
dev_inner_df = pd.read_csv(os.path.join(data_rootpath, 'model_dev_inner_df.csv'))
valid_inner_df = pd.read_csv(os.path.join(data_rootpath, 'model_valid_inner_df.csv'))

In [5]:
# dictionary에 담기
df_dict = dict()

df_dict['dev_inner'] = dev_inner_df
df_dict['valid_inner'] = valid_inner_df

In [6]:
for key, df in df_dict.items():
    print(key, df.shape)

dev_inner (240016, 67)
valid_inner (40942, 67)


In [7]:
df_dict['dev_inner'].head(2)

,no,신청월,직군그룹,근속기간1,웰컴_소득구간,심사원장_소득구간,EXECUTE_FG,DESTRUCT_FG,Y_2017_FG,Y_2018_FG,...,BAD_금융_카드_12,BAD_금융_캐피탈_12,BAD_금융_저축은행_12,BAD_대부_12,BAD_금융_6,BAD_금융_카드_6,BAD_금융_캐피탈_6,BAD_금융_저축은행_6,BAD_대부_6,BAD
0,1,201612,NaN,5.0,5.0,0.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,201612,D,5.0,2.0,0.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# 모형검증데이터 월별 나누기
df_dict['valid_inner_1908'] = valid_inner_df[valid_inner_df['신청월']==201908]
df_dict['valid_inner_1909'] = valid_inner_df[valid_inner_df['신청월']==201909]
df_dict['valid_inner_1910'] = valid_inner_df[valid_inner_df['신청월']==201910]

In [18]:
dev_cb_df = pd.read_csv(os.path.join(data_rootpath, 'model_dev_cb_df.csv'))

In [19]:
valid_cb_df = pd.read_csv(os.path.join(data_rootpath, 'model_valid_cb_df.csv'))

In [20]:
cb_dict = dict()

cb_dict['dev_inner'] = dev_cb_df
cb_dict['valid_inner'] = valid_cb_df

In [21]:
# 모형검증데이터 월별 나누기
cb_dict['valid_cb_1908'] = valid_cb_df[valid_cb_df['신청월']==201908]
cb_dict['valid_cb_1909'] = valid_cb_df[valid_cb_df['신청월']==201909]
cb_dict['valid_cb_1910'] = valid_cb_df[valid_cb_df['신청월']==201910]

# <font color='orange'>02. 주택담보대출 요건별 현황파악

In [9]:
house_cols = df_dict['dev_inner'].columns[df_dict['dev_inner'].columns.str.contains('HOUSE')]

In [10]:
house_cols

Index(['TOTAL_HOUSE_FG_1', 'TOTAL_HOUSE_FG_2', 'TOTAL_HOUSE_FG_3',
       'TOTAL_HOUSE_FG_4', 'BANK_HOUSE_FG', 'INS_HOUSE_FG', 'CARD_HOUSE_FG',
       'LEASE_HOUSE_FG', 'SB_HOUSE_FG', 'COOP_HOUSE_FG', 'CREDUNION_HOUSE_FG',
       'CAP_HOUSE_FG', 'MG_HOUSE_FG', 'ETC_HOUSE_FG', 'ACC_SB_HOUSE_FG',
       'ACC_ML_HOUSE_FG', 'ACC_P2P_HOUSE_FG', 'TOTAL_HOUSE_FG'],
      dtype='object')

In [11]:
# 전체 건수
house_dict = dict()

for key, df in df_dict.items():
    this = pd.DataFrame()
    df = df[df['TOTAL_PRE_BAD_FG'] == 0]
    for col in house_cols:
        value = df[col].sum(axis = 0)
        bad = df[df[col] == 1]['BAD'].sum(axis = 0)
        
        this = this.append(pd.DataFrame({'cnt' : value, 
                                         'bad_cnt' : bad}, index = [0]),
                           ignore_index = True)
        
        
    this.index = house_cols
    house_dict[key] = this

In [13]:
house_dict['dev_inner']

,cnt,bad_cnt
TOTAL_HOUSE_FG_1,11104,4121
TOTAL_HOUSE_FG_2,12909,4688
TOTAL_HOUSE_FG_3,12909,4688
TOTAL_HOUSE_FG_4,12909,4688
BANK_HOUSE_FG,6427,2210
INS_HOUSE_FG,1166,464
CARD_HOUSE_FG,0,0
LEASE_HOUSE_FG,0,0
SB_HOUSE_FG,168,108
COOP_HOUSE_FG,1639,598


In [15]:
# 한 엑셀파일에 여러 sheet로 저장


save_path = os.path.join(r'C:\Users\w10\Desktop\신용평가모형 세미나\2주차\데이터\주택담보대출현황', '요건별_주택담보대출현황_결과.xlsx')
with pd.ExcelWriter(save_path) as writer:
    for key, result in house_dict.items():
        result.to_excel(writer, sheet_name = key, )

# <font color='orange'>03. 주담대 보유 건수 현황

* 기부도 대상 제외

In [16]:
# BAD값이 0 or 1로 설정 필요
def _grouping_grade(data, grade_var, bad_var):
    grp = data.groupby(by = [grade_var], as_index = False).agg(
    {
        bad_var : ('count','sum')
    })
    grp.columns = [grade_var, 'TOTAL_CNT', 'BAD_CNT']
    grp['DISTRIBUTION'] = grp['TOTAL_CNT'] / data.shape[0]
    grp['BAD_RATIO'] = grp['BAD_CNT'] / grp['TOTAL_CNT']
    return grp
    

In [23]:
house_cnt_dict = dict()
for key, inner, cb in zip(df_dict.keys(), df_dict.values(), cb_dict.values()):
    inner = inner[inner['TOTAL_PRE_BAD_FG'] == 0]
    inner = inner[inner['TOTAL_HOUSE_FG'] == 1]
    
    need_no_list = inner['no'].tolist()
    
    cb_df = cb[cb['no'].isin(need_no_list)]
    cb_df['BAD'] = inner['BAD']
    grp = _grouping_grade(cb_df, 'L23001001','BAD',)
    
    house_cnt_dict[key] = grp

In [24]:
house_cnt_dict['dev_inner']

,L23001001,TOTAL_CNT,BAD_CNT,DISTRIBUTION,BAD_RATIO
0,0,1447,607,0.100794,0.419489
1,1,12241,4472,0.852675,0.365330
2,2,615,197,0.042839,0.320325
3,3,47,19,0.003274,0.404255
4,4,6,0,0.000418,0.000000


In [25]:
# 한 엑셀파일에 여러 sheet로 저장


save_path = os.path.join(r'C:\Users\w10\Desktop\신용평가모형 세미나\2주차\데이터\주택담보대출현황', '주택담보대출_보유건수현황_결과.xlsx')
with pd.ExcelWriter(save_path) as writer:
    for key, result in house_cnt_dict.items():
        result.to_excel(writer, sheet_name = key, )

# <font color='orange'> 04. 주담대 업권별 건수 현황

In [26]:
def _pivot_grade(data, grade_var, bad_var, column_var):
    '''
    grade_var = index로 갈 변수
    bad_var = 계산할 변수 
    column_var = column으로 갈 변수
    '''
    piv = data.pivot_table(index = grade_var, columns = column_var, values = [bad_var],
                          aggfunc = ['count','sum'],
                           fill_value = 0,
                          margins = True,
                          margins_name = 'ALL'
                          )
    # 컬럼 정렬
    piv.sort_index( axis = 1, inplace = True, level = column_var)
    # 컬럼명 변경
    piv.columns = ['TOTAL_CNT_' + column_var + '_' + str(col3) if col1 == 'count' else 'BAD_CNT_' + column_var + '_' + str(col3) for (col1, col2, col3) in piv.columns.tolist()]
    # reset index
    piv.reset_index(inplace = True, )
    
    return piv

In [27]:
house_cols = ['LA0001016', 'LA0005008', 'LA0006011', 'LA0007201', 'LA0008008',
              'LA0012005', 'LA0012208',
             'LA0014008', 'LA0029203', 'LA0099252',
             'LU0021006_1', 'LU0024013_1', 'LU0025004_1', ]

In [28]:
busi_cols = ['은행','보험','카드','리스렌탈','저축은행','조합업계',
            '신협','캐피탈','새마을금고','기타','계좌별_저축은행', '대부업','P2P연계대부업']

In [33]:
house_busi_dict = dict()

for key, inner, cb in zip(df_dict.keys(), df_dict.values(), cb_dict.values()):
    inner = inner[inner['TOTAL_PRE_BAD_FG'] == 0]
    inner = inner[inner['TOTAL_HOUSE_FG'] == 1]
    
    need_no_list = inner['no'].tolist()
    
    cb_df = cb[cb['no'].isin(need_no_list)]
    cb_df['BAD'] = inner['BAD']
    
    for col, name in zip(house_cols, busi_cols):
        grp = _grouping_grade(cb_df, col, 'BAD')
        # 4이상은 합쳐서
        vv = grp[grp[col] >= 4].sum()
        vv = pd.DataFrame(vv).transpose()
        grp = grp[grp[col] < 4]
        grp = grp.append(vv, ignore_index = True)
        
        house_busi_dict[key + '_' + name] = grp

In [34]:
house_busi_dict['dev_inner_은행']

,LA0001016,TOTAL_CNT,BAD_CNT,DISTRIBUTION,BAD_RATIO
0,0.0,7929.0,3085.0,0.552313,0.389078
1,1.0,5206.0,1769.0,0.362636,0.339800
2,2.0,828.0,318.0,0.057676,0.384058
3,3.0,215.0,69.0,0.014976,0.320930
4,52.0,178.0,54.0,0.012399,2.305454


In [35]:
# 한 엑셀파일에 여러 sheet로 저장

save_path = os.path.join(r'C:\Users\w10\Desktop\신용평가모형 세미나\2주차\데이터\주택담보대출현황', '요건별_업권별_주택담보대출현황_결과.xlsx')
with pd.ExcelWriter(save_path) as writer:
    for key, result in house_busi_dict.items():
        result.to_excel(writer, sheet_name = key, )

In [36]:
for key, df in df_dict.items():
    a = df[df['TOTAL_PRE_BAD_FG']==0]
    print(key, a.shape)

dev_inner (198719, 67)
valid_inner (35879, 67)
valid_inner_1908 (13068, 67)
valid_inner_1909 (11984, 67)
valid_inner_1910 (10827, 67)
